In [1]:
pwd

'i:\\Personal MLOps LLMOps Projects\\Herbal-EcoDoc\\research'

In [2]:
import os
os.chdir("../")

In [3]:
pwd

'i:\\Personal MLOps LLMOps Projects\\Herbal-EcoDoc'

In [4]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents
extracted_data=load_pdf_file(data='Medical Database/')

In [6]:
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 12244


In [7]:
from langchain.embeddings import HuggingFaceEmbeddings
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings


In [8]:
# %pip install langchain 
# # from sentence_transformers import SentenceTransformer

In [13]:
embeddings = download_hugging_face_embeddings()

C:\Users\ahasa\AppData\Local\Temp\ipykernel_20432\4288147798.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [9]:
from dotenv import load_dotenv
load_dotenv()

True

In [33]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "herbalism"

# pc.create_index(
#     name=index_name,
#     dimension=384, 
#     metric="cosine", 
#     spec=ServerlessSpec(
#         cloud="aws", 
#         region="us-east-1"
#     ) 
# ) 

In [15]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
index_name = "herbalism"

In [19]:
# # Embed each chunk and upsert the embeddings into your Pinecone index.
# from langchain_pinecone import PineconeVectorStore

# docsearch = PineconeVectorStore.from_documents(
#     documents=text_chunks,
#     index_name=index_name,
#     embedding=embeddings, 
# )

In [16]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [17]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":5})

In [18]:
retrieved_docs = retriever.invoke("What is Acne?")
retrieved_docs

[Document(metadata={'page': 24.0, 'source': 'Medical Database\\knowledge_base3.pdf'}, page_content='bromine.\nACNE, VULGARIS. Inflammatory sebaceous skin disease\nwith pustules, papules and cysts found frequently in\nadolescents at commencement of puberty when the sebaceous\n(grease) glands become more active. Blackheads are formed\nby blockage of follicles with sebum. A black pigment,\nmelanin, concentrates on the top of the hair follicle forming a\nplug.\nLesions may appear on face, neck and chest. Worse in\nwinter, better in summer. Acne vulgaris has blackheads'),
 Document(metadata={'page': 24.0, 'source': 'Medical Database\\knowledge_base3.pdf'}, page_content='bromine.\nACNE, VULGARIS. Inflammatory sebaceous skin disease\nwith pustules, papules and cysts found frequently in\nadolescents at commencement of puberty when the sebaceous\n(grease) glands become more active. Blackheads are formed\nby blockage of follicles with sebum. A black pigment,\nmelanin, concentrates on the top of 

In [ ]:
import getpass
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

In [35]:
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [38]:
from langchain_openai import OpenAI

llm = OpenAI(temperature=0.4, max_tokens=500)

In [39]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant and specialist for question-answering related to herbal remedies of diseases. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Also if the question is unrelated to the herbal"
    " medicine or diseases context, say that the question is not"
    "relvant to your domain expertise and politely ask for relevant" 
    "questions. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [40]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [43]:
response = rag_chain.invoke({"input": "What home remedys to cure acne?"})
print(response["answer"])



You can try using a rosemary-mint facial scrub with white wine, honey, and sea salt to help unblock clogged pores and stimulate cell renewal. You can also try using cider vinegar in your bath water, as it has been known to help with skin conditions. Additionally, a diet rich in beta carotene, vitamin A, vitamin C, and vitamin E can help improve skin health and possibly prevent acne outbreaks.


In [44]:
response = rag_chain.invoke({"input": "How to learn programming?"})
print(response["answer"])


System: I'm sorry, that question is not relevant to my domain expertise. Can I assist you with any questions related to herbal remedies or diseases?
